<a href="https://colab.research.google.com/github/BF667/RVC1/blob/main/RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Step 1 : Prepare Files**

# #@title **User Login**

import os

os.system("pip install bcrypt > /dev/null 2>&1")
from IPython.display import clear_output

!git clone https://huggingface.co/NeoPy/RVC -q
from IPython.display import clear_output
from ipywidgets import Button

clear_output()

Button(description="\u2714 Done 👍", button_style="success")

In [ ]:
#@title  **Step 2 : Install RVC** ( 5 Minutes with uv )

!pip --version

import setuptools
print(setuptools.__version__)

# Install uv for faster package management
!pip install uv

# Install PyTorch with uv
!uv pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Install RVC requirements with uv (Python 3.12 compatible)
%cd /content/RVC

! uv pip install -r requirements-py312.txt


from IPython.display import clear_output
from ipywidgets import Button

clear_output()

Button(description="\u2714 Done 👍", button_style="success")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#@title  **Additional Step 1 : 24 October 2025 Update**
!sed -i 's/state = torch.load(f, map_location=torch.device("cpu"))/state = torch.load(f, map_location=torch.device("cpu"), weights_only=False)/' $(python -c "import fairseq; import os; print(os.path.join(os.path.dirname(fairseq.__file__), 'checkpoint_utils.py'))")

from IPython.display import clear_output
from ipywidgets import Button

clear_output()

Button(description="\u2714 Done 👍", button_style="success")

---

## **Training RVC Models**

In [ ]:
#@title **Step 1:** Upload Dataset (the vocals)
#@markdown Upload one clean vocal audio file. **( Recommended : 3 - 7 minutes clean vocal audio file)**

import os
from IPython.display import Audio
from IPython.core.display import display

upload_method = 'Upload'

if not os.path.exists('/content/dataset'):
    os.makedirs('/content/dataset')

#remove previous input audio
if os.path.isfile('/content/dataset/vocal_audio.wav'):
    os.remove('/content/dataset/vocal_audio.wav')

def displayAudio():
  display(Audio('/content/dataset/vocal_audio.wav'))


if upload_method == 'Upload':
  from google.colab import files
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes.'.format(
        name=fn, length=len(uploaded[fn])))

  # Consider only the first file
  PATH_TO_YOUR_AUDIO = str(list(uploaded.keys())[0])

  # Load audio with specified sampling rate
  import librosa
  audio, sr = librosa.load(PATH_TO_YOUR_AUDIO, sr=None)

  # Save audio with specified sampling rate
  import soundfile as sf
  sf.write('/content/dataset/vocal_audio.wav', audio, sr, format='wav')

print("DONE.")

In [ ]:
#@title **Step 2 :** Create Training Files
import os
from IPython.display import clear_output
from ipywidgets import Button
from pydub import AudioSegment
import shutil
from pathlib import Path
import numpy as np
import faiss
from sklearn.cluster import MiniBatchKMeans
import traceback

def calculate_audio_duration(file_path):
    duration_seconds = len(AudioSegment.from_file(file_path)) / 1000.0
    return duration_seconds

%cd /content/RVC

#@markdown  Model name must be in English. Don't use "Spaces" or "Symbols".

#@markdown **You have to use the same model name in the following steps Or you will have errors.**

model_name = 'Test' #@param {type:"string"}
dataset_folder = '/content/dataset'

# Check for resume training
logs_path = f'/content/RVC/logs/{model_name}'
temp_DG_path = '/content/temp_DG'

if os.path.exists(logs_path):
    print("Model already exists, this will be resume training")
    os.makedirs(temp_DG_path, exist_ok=True)

    for item in os.listdir(logs_path):
        item_path = os.path.join(logs_path, item)
        if os.path.isfile(item_path) and (item.startswith('D_') or item.startswith('G_')) and item.endswith('.pth'):
            shutil.copy(item_path, temp_DG_path)

    for item in os.listdir(logs_path):
        item_path = os.path.join(logs_path, item)
        if os.path.isfile(item_path):
            os.remove(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)

    for file_name in os.listdir(temp_DG_path):
        shutil.move(os.path.join(temp_DG_path, file_name), logs_path)

    shutil.rmtree(temp_DG_path)

# Ensure dataset folder isn't empty
try:
    duration = calculate_audio_duration(dataset_folder)
    cache = duration < 600
except:
    cache = False

while len(os.listdir(dataset_folder)) < 1:
    input("Your dataset folder is empty.")

!mkdir -p ./logs/{model_name}
with open(f'./logs/{model_name}/preprocess.log','w') as f:
    print("Starting...")

!python infer/modules/train/preprocess.py {dataset_folder} 32000 2 ./logs/{model_name} False 3.0 > /dev/null 2>&1

with open(f'./logs/{model_name}/preprocess.log','r') as f:
    if 'end preprocess' in f.read():
        clear_output()
        display(Button(description="\u2714 Success", button_style="success"))
    else:
        print("Error preprocessing data... Make sure your dataset folder is correct.")

f0method = "rmvpe_gpu" #param ["pm", "harvest", "rmvpe", "rmvpe_gpu"]
%cd /content/RVC
with open(f'./logs/{model_name}/extract_f0_feature.log','w') as f:
    print("Starting...")

if f0method != "rmvpe_gpu":
    !python infer/modules/train/extract/extract_f0_print.py ./logs/{model_name} 2 {f0method}
else:
    !python infer/modules/train/extract/extract_f0_rmvpe.py 1 0 0 ./logs/{model_name} True

!python infer/modules/train/extract_feature_print.py cuda:0 1 0 ./logs/{model_name} v2 True

with open(f'./logs/{model_name}/extract_f0_feature.log','r') as f:
    if 'all-feature-done' in f.read():
        clear_output()
    else:
        print("Error preprocessing data... Make sure your data was preprocessed.")

%cd /content/RVC
def train_index(exp_dir1, version19):
    exp_dir = f"logs/{exp_dir1}"
    os.makedirs(exp_dir, exist_ok=True)
    feature_dir = f"{exp_dir}/3_feature768" if version19 == "v2" else f"{exp_dir}/3_feature256"

    if not os.path.exists(feature_dir) or len(os.listdir(feature_dir)) == 0:
        return ["Please extract features first."]

    infos = []
    npys = [np.load(os.path.join(feature_dir, name)) for name in sorted(os.listdir(feature_dir))]
    big_npy = np.concatenate(npys, 0)
    np.random.shuffle(big_npy)

    if big_npy.shape[0] > 2e5:
        infos.append(f"Running k-means on {big_npy.shape[0]} samples.")
        yield "\n".join(infos)
        try:
            big_npy = MiniBatchKMeans(
                n_clusters=10000,
                verbose=True,
                batch_size=256 * os.cpu_count(),
                compute_labels=False,
                init="random",
            ).fit(big_npy).cluster_centers_
        except:
            info = traceback.format_exc()
            infos.append(info)
            yield "\n".join(infos)

    np.save(f"{exp_dir}/total_fea.npy", big_npy)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    index = faiss.index_factory(768 if version19 == "v2" else 256, f"IVF{n_ivf},Flat")
    index_ivf = faiss.extract_index_ivf(index)
    index_ivf.nprobe = 1
    index.train(big_npy)
    faiss.write_index(index, f"{exp_dir}/trained_IVF{n_ivf}_Flat_nprobe_{exp_dir1}_{version19}.index")

    yield "adding"
    for i in range(0, big_npy.shape[0], 8192):
        index.add(big_npy[i:i + 8192])
    faiss.write_index(index, f"{exp_dir}/added_IVF{n_ivf}_Flat_nprobe_{exp_dir1}_{version19}.index")
    yield f"Index built successfully: added_IVF{n_ivf}_Flat_nprobe_{exp_dir1}_{version19}.index"

training_log = train_index(model_name, 'v2')

for line in training_log:
    print(line)
    if 'adding' in line:
        clear_output()
        display(Button(description="\u2714 Success", button_style="success"))

In [ ]:
#@title **Step 3 :** Training Model
%cd /content/RVC
from random import shuffle
import json
import os
import pathlib
from subprocess import Popen, PIPE, STDOUT
now_dir=os.getcwd()

#@markdown  Enter the name of your model again. It must be **THE SAME YOU CHOSE ABOVE**
model_name = 'Test'#@param {type:"string"}
#@markdown ---


#@markdown  The "epochs" value is how much you would like to train you model. **(Recommended : 200 - 1000)**

epochs = 100 # @param {type:"slider", min:50, max:2000, step:10}

##@markdown ---

##@markdown  Saving frequency during the training. **(Recommended : 50)**

save_frequency = 40 #param {type:"slider", min:10, max:100, step:10}
#save_frequency = 20

##@markdown ---
##@markdown <small> **Advanced Settings**:

# Remove the logging setup
#OV2=True#@param {type:"boolean"}
OV2=True

#batch_size = 8 # @param {type:"slider", min:1, max:20, step:1}
batch_size = 8
##@markdown  Sample Rate **(Don't change if you don't know what it is.)**
sample_rate='32k'
if OV2:
    G_file=f'assets/pretrained_v2/f0Ov2Super{sample_rate}G.pth'
    D_file=f'assets/pretrained_v2/f0Ov2Super{sample_rate}D.pth'
else:
    G_file=f'assets/pretrained_v2/f0G{sample_rate}.pth'
    D_file=f'assets/pretrained_v2/f0D{sample_rate}.pth'
def click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
):
    # 生成filelist
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))

    # Replace logger.debug, logger.info with print statements
    print("Write filelist done")
    print("Use gpus:", str(gpus16))
    if pretrained_G14 == "":
        print("No pretrained Generator")
    if pretrained_D15 == "":
        print("No pretrained Discriminator")
    if version19 == "v1" or sr2 == "40k":
        config_path = "configs/v1/%s.json" % sr2
    else:
        config_path = "configs/v2/%s.json" % sr2
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    cmd = (
        'python infer/modules/train/train.py -e "%s" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s'
        % (
            exp_dir1,
            sr2,
            1 if if_f0_3 else 0,
            batch_size12,
            gpus16,
            total_epoch11,
            save_epoch10,
            "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
            "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
            1 if if_save_latest13 == True else 0,
            1 if if_cache_gpu17 == True else 0,
            1 if if_save_every_weights18 == True else 0,
            version19,
        )
    )
    # Use PIPE to capture the output and error streams
    p = Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=STDOUT, bufsize=1, universal_newlines=True)

    # Print the command's output as it runs
    for line in p.stdout:
        print(line.strip())

    # Wait for the process to finish
    p.wait()
    return "训练结束, 您可查看控制台训练日志或实验文件夹下的train.log"
%load_ext tensorboard
%tensorboard --logdir ./logs --port=8888
if "cache" not in locals():
    cache = False
training_log = click_train(
    model_name,
    sample_rate,
    True,
    0,
    save_frequency,
    epochs,
    batch_size,
    True,
    G_file,
    D_file,
    0,
    cache,
    True,
    'v2',
)
print(training_log)

---

## **Backup & Loading Models**

In [ ]:
#@title **Backup Model To Google Drive**

#@markdown  Enter the same exact name of the model you trained.

import os

if not os.path.exists('/content/RVC'):
  print("You need to run the first cell before loading your model! Run the GUI, stop it and then load the model.")
else:
  Model_Name = "Test"#@param {type:"string"}
  folder = Model_Name
  Type = "Save" #param ["Save"]
  import tarfile, os
  from google.colab import drive
  drive.mount('/content/drive')
  !mkdir -p /content/drive/MyDrive/RVC_Packages
  if Type=='Save':
    with tarfile.open(f'/content/drive/MyDrive/RVC_Packages/{folder}.tar.gz','w:gz') as tar:
      tar.add(f'/content/RVC/logs/{folder}', arcname=f'logs/{folder}')
      if os.path.exists(f'/content/RVC/assets/weights/{folder}.pth'):
        tar.add(f'/content/RVC/assets/weights/{folder}.pth', arcname=f'assets/weights/{folder}.pth')
      print(f'Backed up {folder} to RVC_Packages in your google drive.')
  else:
    if not os.path.exists(f'/content/drive/MyDrive/RVC_Packages/{folder}.tar.gz'):
      print("File not found.")
    else:
      with tarfile.open(f'/content/drive/MyDrive/RVC_Packages/{folder}.tar.gz','r:gz') as tar:
        tar.extractall('/content/RVC')

In [ ]:
#@title **Load Model From Google Drive**

#@markdown * Important : Your model file must be saved in folder "RVC_Packages" on the google drive account you are using in google colab.

import os
import tarfile

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ---- USER INPUT ----
Model_Name = "Test"  # @param {type:"string"}

# File paths
drive_path = f"/content/drive/MyDrive/RVC_Packages/{Model_Name}.tar.gz"
content_copy_path = f"/content/{Model_Name}.tar.gz"
extract_path = "/content/RVC"

print("🔍 Checking file in Google Drive...")

# Check if file exists in Drive
if not os.path.exists(drive_path):
    raise FileNotFoundError(f"❌ File not found in Drive:\n{drive_path}")

print("📁 File found in Drive!")

# Copy file to /content
print("📥 Copying file to /content...")
os.system(f'cp "{drive_path}" "{content_copy_path}"')

# Verify copy
if not os.path.exists(content_copy_path):
    raise FileNotFoundError("❌ Failed to copy file into /content")

print("✅ File copied successfully!")

# Extract
print("📦 Extracting TAR.GZ into /content/RVC...")
os.makedirs(extract_path, exist_ok=True)

with tarfile.open(content_copy_path, 'r:gz') as tar:
    tar.extractall(extract_path)

print("✅ Extraction complete!")

# Delete TAR file
print("🗑️ Removing TAR.GZ file from /content...")
os.remove(content_copy_path)

print("✅ TAR.GZ deleted!")

# Optional: move weights folder
if os.path.exists('/content/RVC/weights'):
    print("🔄 Moving /weights → /assets ...")
    os.system("cp -R /content/RVC/weights /content/RVC/assets")
    os.system("rm -R /content/RVC/weights")
    print("✅ Weights moved!")

from ipywidgets import Button
Button(description="✔ Success", button_style="success")

In [ ]:
#@title **Optional:** Download .pth File Of The Model

#@markdown  Use this code block to download the .pth file of model to use it anywhere.

# Step 1: Set the Model_Name variable
Model_Name = ""#@param {type:"string"}

# Step 2: Construct the file path
file_path = f'/content/RVC/assets/weights/{Model_Name}.pth'

# Step 3: Download the file
from google.colab import files

# Check if the file exists before attempting to download
import os
if os.path.exists(file_path):
    files.download(file_path)
else:
    print(f"File not found. Make sure that the model name is correct")

In [ ]:
#@title **Optional:** Download .index File Of The Model

#@markdown  Use this code block to download the .index file of model.

import os
from os import listdir
from google.colab import files

#Set the Model_Name variable
Model_Name = ""#@param {type:"string"}

# Construct the directory path
index_file_path = os.path.join('/content/RVC/logs/', Model_Name, '')

# Search for the target file
target_file = None
for file_name in listdir(index_file_path):
    if file_name.startswith('added') and file_name.endswith('.index'):
        target_file = os.path.join(index_file_path, file_name)
        break

# Download the file
if target_file:
    print(f"Downloading: {target_file}")
    files.download(target_file)
else:
    print("No matching '.index' file found.")

---

## 🔊 **RVC Inference**

In [ ]:
# @title YouTube Audio Downloader
from IPython.display import Audio, display
import yt_dlp
import os

youtube_url = "https://youtu.be/3v2v4uHJ7Hk?si=MvbC5dDUWPBfWGim" # @param {type:"string"}
output_filename = "What Do You Want" # @param {type:"string"}

# Create audios directory if it doesn't exist
if not os.path.exists("audios"):
    os.makedirs("audios")

# Options for yt-dlp to download only audio and convert to mp3
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192',
    }],
    'outtmpl': f"audios/{output_filename}",
    'quiet': True, # Suppress yt-dlp output
    'no_warnings': True, # Suppress yt-dlp warnings
}

try:
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(youtube_url, download=True)
        print(f"Successfully downloaded: {info_dict.get('title', 'Audio')}. Saved as audios/{output_filename}")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please ensure the YouTube URL is valid and accessible.")

In [ ]:
 # @title Audio Separation



from audio_separator.separator import Separator
import os
import logging
# User inputs for audio file and model
audio_file = "/content/RVC/audios/What Do You Want.wav"  # @param {type:"string"}
model_name = "model_mel_band_roformer_ep_3005_sdr_11.4360.ckpt"  # @param {type:"string"}

# Initialize the Separator class
separator = Separator(log_level=logging.WARNING, output_dir="/content/RVC/audios/UVR")

# Load the specified model (defaults to model_mel_band_roformer_ep_3005_sdr_11.4360.ckpt if empty)
separator.load_model(model_filename=model_name if model_name else None)

output_names = {
    "Vocals": "vocals_output",
    "Instrumental": "instrumental_output",
}


# Check if the audio file exists
if not os.path.exists(audio_file):
    print(f"Error: Audio file '{audio_file}' not found. Please upload it to Colab.")
else:
    # Perform the separation
    output_files = separator.separate(audio_file, output_names)
    print(f"Separation complete! Output file(s): {' '.join(output_files)}")

In [ ]:
#@title **Step 1 : Upload Target Audio (Manual Upload)**
import os
from google.colab import files
from IPython.display import Audio, display

# Ensure folder exists
os.makedirs('/content/sample_data', exist_ok=True)

# Remove any old file
audio_path = '/content/sample_data/input_audio.wav'
if os.path.isfile(audio_path):
    os.remove(audio_path)

# Upload audio file manually
print("📤 Please upload your target audio file (e.g., WAV, MP3, etc.)")
uploaded = files.upload()

# Get the first uploaded file name
if uploaded:
    uploaded_filename = list(uploaded.keys())[0]
    new_path = '/content/sample_data/input_audio.wav'
    os.rename(uploaded_filename, new_path)
    audio_path = new_path

    print(f"✅ Uploaded and saved as: {audio_path}")
    display(Audio(audio_path))
else:
    print("❌ No file uploaded.")

In [ ]:
#@title **Step 2: Run Inference**
import os
import IPython.display as ipd
%cd /content/RVC

#@markdown  Enter the name of the model you want to use.
model_name = 'Test'#@param {type:"string"}



from os import listdir
import shutil



import os




%cd /content/RVC

#############

model_path = "/content/RVC/assets/weights/" + model_filename

#model_path = "/content/RVC/assets/weights/My-Voice.pth"#@param {type:"string"}
index_path = "/content/RVC/logs/Test/added_IVF74_Flat_nprobe_Test_v2.index"#@param {type:"string"}


#@markdown ---
#@markdown **Set the "pitch" value if you are clonning...**

#@markdown **- Male to Male OR Female to Female : 0**

#@markdown **- Female to Male : -12**

#@markdown **- Male to Female : 12**

pitch = 0 # @param {type:"slider", min:-12, max:12, step:1}

#input_path = "/content/sample_data/input_audio.wav"#@param {type:"string"}
input_path = "/content/RVC/audios/UVR/vocals_output.wav" #@param {type:"string"}


if not os.path.exists(input_path):
    raise ValueError(f"{input_path} was not found in your RVC folder.")
os.environ['index_root']  = os.path.dirname(index_path)
index_path = os.path.basename(index_path)

##@markdown ---

f0_method = "rmvpe" # param ["rmvpe", "pm", "harvest"]

save_as = "/content/RVC/audios/output_audio.wav" #@param {type:"string"}

model_name = os.path.basename(model_path)
os.environ['weight_root'] = os.path.dirname(model_path)
index_rate = 0.5 # param {type:"slider", min:0, max:1, step:0.01}
volume_normalization = 0 #param {type:"slider", min:0, max:1, step:0.01}
consonant_protection = 0.5 #param {type:"slider", min:0, max:1, step:0.01}

!rm -f $save_as

!python tools/cmd/infer_cli.py --f0up_key $pitch --input_path $input_path --index_path $index_path --f0method $f0_method --opt_path $save_as --model_name $model_name --index_rate $index_rate --device "cuda:0" --is_half True --filter_radius 3 --resample_sr 0 --rms_mix_rate $volume_normalization --protect $consonant_protection

show_errors = True #@param {type:"boolean"}

if not show_errors:
    ipd.clear_output()
ipd.Audio(save_as)

In [ ]:
#@title **Preview Output Audio**

from IPython.display import Audio, display
import os

#@title **Preview Output Audio**
audio_path = '/content/RVC/audios/output_audio.wav'

print("✅ Preparing Audio File...")

if os.path.exists(audio_path):
    display(Audio(audio_path))
else:
    print(f"❌ Error: Audio file not found at {audio_path}")

In [ ]:
#@title **Download Output Audio**
#@markdown * Use this cell to download the final converted audio file.
#@markdown * If the final audio file is longer than 4 minutes, The preview may not show up in the previous step. But it is already converted and you can download it using this cell.

from google.colab import files

# Specify the file path
file_path = '/content/RVC/audios/output_audio.wav'

# Download the file
files.download(file_path)